<h1>Let's try to create a model that identifies fake and real news.</h1><br>
the dataset will be https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset

The idea is to train a separate model for fake news and real news and then combine them.

In [7]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import spacy

df_fake=pd.read_csv('..//Datasets/Fake_news.csv')
df_real=pd.read_csv('..//Datasets/Real_news.csv')

df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


<h2>Preprocessing .<br>


In [8]:

# function to remove links and special chars, also removing extra whitespaces.
def clean_text(text):  
    # remove links by substituting target words with nothing
    text=re.sub(r'http\S+|www\S+|https\S+', '',text)
    # remove special chars
    text=re.sub(r'[^a-zA-ZÀ-ú\s]','',text)
    # tokenization
    tokens=text.split()
    # remove extra whitespace
    tokens=[token.strip() for token in tokens]
    cleaned_text=' '.join(tokens)
    return cleaned_text

nlp=spacy.load('en_core_web_sm')
def possifier(text_df):
    pos_tags_list = []
    for text in text_df:
        doc = nlp(text)
        pos_tags = [(token.pos_) for token in doc]
        pos_tags_list.append(pos_tags)
    return pos_tags_list

# adding pos_tag counts might help the model further
def process_pos_tags(tags):
    pos_counts = {}
    for tag in tags:
        if tag in pos_counts:
            pos_counts[tag] += 1
        else:
            pos_counts[tag] = 1
    return pos_counts

In [9]:
#df_fake['text']=[clean_text(text) for text in df_fake['text']]
#df_real['text']=[clean_text(text) for text in df_real['text']]
df_fake['pos counts'] = df_fake['text'].apply(process_pos_tags)
df_fake.pos_counts_per_row.head()

0    {'D': 21, 'o': 142, 'n': 140, 'a': 182, 'l': 9...
1    {'H': 5, 'o': 125, 'u': 45, 's': 92, 'e': 173,...
2    {'O': 7, 'n': 137, ' ': 599, 'F': 9, 'r': 173,...
3    {'O': 8, 'n': 97, ' ': 474, 'C': 9, 'h': 104, ...
4    {'P': 7, 'o': 155, 'p': 36, 'e': 257, ' ': 433...
Name: pos_counts_per_row, dtype: object

In this case, I have no target data

In [10]:
# training data
X=df['message']

# Target is the sentiment so y=sentiment
y=df['sentiment']

cv=CountVectorizer()
X=cv.fit_transform(X)

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=0)

le=LabelEncoder()
y_test_encoded=le.fit_transform(y_test)

NameError: name 'df' is not defined

In [ ]:
# Let's try XGBoost
# Unlike MultinomialNB, XGBClassifier demands all data to be numerical.
# So let's do that by using what we used previously, the label encoder
y_train_encoded=le.fit_transform(y_train)

# Converting X_train and X_test to dense arrays in order for them to work with XGBoost
X_train_dense=X_train.toarray()
X_test_dense=X_test.toarray()

# one hot encoding categorical values
encoder=OneHotEncoder(handle_unknown='ignore')
encoder.fit(X_train_dense)
X_train_encoded=encoder.transform(X_train_dense)
X_test_encoded=encoder.transform(X_test_dense)

# Initiating regressor...I mean classifier, so far n_estimators=300 and learning_rate=0.3 seem the best
xgb=XGBClassifier(n_estimators=1000,learning_rate=0.25,early_stopping_rounds=5)

# Fitting model, in order to use early_stopping_rounds we need to add an eval_set to the fitting part.
xgb.fit(X_train_encoded,y_train_encoded, eval_set=[(X_test_encoded, y_test_encoded)], verbose=True)

# Predicting
pred_xgb_split=xgb.predict(X_test_encoded)

# Convert numerical predictions back to categorical labels
y_pred_labels = le.inverse_transform(pred_xgb_split)

[0]	validation_0-mlogloss:0.97773
[1]	validation_0-mlogloss:0.88978
[2]	validation_0-mlogloss:0.83131
[3]	validation_0-mlogloss:0.78910
[4]	validation_0-mlogloss:0.74464
[5]	validation_0-mlogloss:0.72004
[6]	validation_0-mlogloss:0.69919
[7]	validation_0-mlogloss:0.67303
[8]	validation_0-mlogloss:0.65912
[9]	validation_0-mlogloss:0.64356
[10]	validation_0-mlogloss:0.63320
[11]	validation_0-mlogloss:0.61390
[12]	validation_0-mlogloss:0.60269
[13]	validation_0-mlogloss:0.59335
[14]	validation_0-mlogloss:0.58258
[15]	validation_0-mlogloss:0.57563
[16]	validation_0-mlogloss:0.56660
[17]	validation_0-mlogloss:0.55860
[18]	validation_0-mlogloss:0.55364
[19]	validation_0-mlogloss:0.54344
[20]	validation_0-mlogloss:0.53658
[21]	validation_0-mlogloss:0.53227
[22]	validation_0-mlogloss:0.52762
[23]	validation_0-mlogloss:0.52534
[24]	validation_0-mlogloss:0.52158
[25]	validation_0-mlogloss:0.51799
[26]	validation_0-mlogloss:0.51574
[27]	validation_0-mlogloss:0.51042
[28]	validation_0-mlogloss:0.5

In [ ]:
# The best hyperparams seem to be n_estimators=1000,learning_rate=0.25 where early_stopping_rounds=5. Therefore i shall train an army of forces to defeat Sauron. 
xgb_full=XGBClassifier(n_estimators=1000,learning_rate=0.25)
# The full data X must be encoded since it is not categorical, while the target data y is.
encoder.fit(X.toarray())
X_encoded = encoder.transform(X.toarray())
y_encoded=le.fit_transform(y)
xgb_full.fit(X_encoded,y_encoded)

# Predicting
pred_xgb_full = xgb_full.predict(X_encoded)

# Convert numerical predictions back to categorical labels
y_pred_labels_full = le.inverse_transform(pred_xgb_full)


In [ ]:
accuracy = accuracy_score(y_test, y_pred_labels)
mse = mean_squared_error(y_test_encoded, pred_xgb_split)
mae = mean_absolute_error(y_test_encoded, pred_xgb_split)

accuracyf = accuracy_score(y, y_pred_labels_full)
msef = mean_squared_error(y_encoded, pred_xgb_full)
maef = mean_absolute_error(y_encoded, pred_xgb_full)

print(f'Accuracy for the split model: {accuracy}')
print(f'Mean Squared Error for the split model: {mse}')
print (f'Mean Absolute Error for the split model: {mae}')
print(f'Accuracy for the full model: {accuracyf}')
print(f'Mean Squared Error for the full model: {msef}')
print (f'Mean Absolute Error for the full model: {maef}')

Accuracy for the split model: 0.8125
Mean Squared Error for the split model: 0.5965909090909091
Mean Absolute Error for the split model: 0.32386363636363635
Accuracy for the full model: 0.9691780821917808
Mean Squared Error for the full model: 0.08732876712328767
Mean Absolute Error for the full model: 0.04965753424657534
